In [1]:
import pandas as pd
import numpy as np
import warnings, random
warnings.filterwarnings(action='ignore')
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from category_encoders.ordinal import OrdinalEncoder
from sklearn.model_selection import StratifiedKFold

from sklearn.cluster import KMeans
from catboost import CatBoostClassifier, Pool

from sklearn.model_selection import StratifiedKFold,train_test_split


# Load Data

In [2]:
train=pd.read_csv('train2.csv')
test=pd.read_csv('test2.csv')

In [3]:
train.head()

,Unnamed: 0,id,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,...,email,occyp_type,family_size,begin_month,credit,YEARS_BIRTH,Age_group,YEARS_EMPLOYED,EMPLOYED_group,income_avg
0,0,F138994709202500,F,N,N,0,3948.75,Commercial associate,Higher education,Married,...,0,NaN,2.0,6.0,1.0,39,30,12,10~20,1974.375
1,1,F113801540247500,F,N,Y,1,4826.25,Commercial associate,Secondary / secondary special,Civil marriage,...,1,Laborers,3.0,5.0,1.0,32,30,4,1~5,1608.750
2,2,M190874434450000,M,Y,Y,0,8775.00,Working,Higher education,Married,...,0,Managers,2.0,22.0,2.0,53,50,12,10~20,4387.500
3,3,F150882092202500,F,N,Y,0,3948.75,Commercial associate,Secondary / secondary special,Married,...,0,Sales staff,2.0,37.0,0.0,42,40,5,5~10,1974.375
4,4,F150372105157500,F,Y,Y,0,3071.25,State servant,Higher education,Married,...,0,Managers,2.0,26.0,2.0,42,40,5,5~10,1535.625


In [4]:
train.drop(['Unnamed: 0'],axis=1,inplace=True)
train.head()

,id,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,...,email,occyp_type,family_size,begin_month,credit,YEARS_BIRTH,Age_group,YEARS_EMPLOYED,EMPLOYED_group,income_avg
0,F138994709202500,F,N,N,0,3948.75,Commercial associate,Higher education,Married,Municipal apartment,...,0,NaN,2.0,6.0,1.0,39,30,12,10~20,1974.375
1,F113801540247500,F,N,Y,1,4826.25,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,...,1,Laborers,3.0,5.0,1.0,32,30,4,1~5,1608.750
2,M190874434450000,M,Y,Y,0,8775.00,Working,Higher education,Married,House / apartment,...,0,Managers,2.0,22.0,2.0,53,50,12,10~20,4387.500
3,F150882092202500,F,N,Y,0,3948.75,Commercial associate,Secondary / secondary special,Married,House / apartment,...,0,Sales staff,2.0,37.0,0.0,42,40,5,5~10,1974.375
4,F150372105157500,F,Y,Y,0,3071.25,State servant,Higher education,Married,House / apartment,...,0,Managers,2.0,26.0,2.0,42,40,5,5~10,1535.625


In [5]:
test.drop(['Unnamed: 0'],axis=1,inplace=True)
test.head()

,id,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,...,phone,email,occyp_type,family_size,begin_month,YEARS_BIRTH,Age_group,YEARS_EMPLOYED,EMPLOYED_group,income_avg
0,M219900112500,M,Y,N,0,2193.750,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,...,1,0,No work experience,2.0,60.0,61,60,No work experience,No work experience,1096.875
1,F189648671135000,F,N,Y,0,2632.500,State servant,Higher education,Married,House / apartment,...,1,0,Core staff,2.0,36.0,52,50,23,20~30,1316.250
2,F1588721769372,F,N,Y,0,1352.754,Working,Secondary / secondary special,Married,House / apartment,...,1,0,Laborers,2.0,40.0,44,40,0,~1,676.377
3,M192702531112500,M,Y,N,0,2193.750,Commercial associate,Secondary / secondary special,Married,House / apartment,...,0,0,Drivers,2.0,41.0,53,50,6,5~10,1096.875
4,F178229385225000,F,Y,Y,0,4387.500,State servant,Higher education,Married,House / apartment,...,0,0,Managers,2.0,8.0,49,40,25,20~30,2193.750


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26440 entries, 0 to 26439
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              26440 non-null  object 
 1   gender          26440 non-null  object 
 2   car             26440 non-null  object 
 3   reality         26440 non-null  object 
 4   child_num       26440 non-null  int64  
 5   income_total    26440 non-null  float64
 6   income_type     26440 non-null  object 
 7   edu_type        26440 non-null  object 
 8   family_type     26440 non-null  object 
 9   house_type      26440 non-null  object 
 10  work_phone      26440 non-null  int64  
 11  phone           26440 non-null  int64  
 12  email           26440 non-null  int64  
 13  occyp_type      22707 non-null  object 
 14  family_size     26440 non-null  float64
 15  begin_month     26440 non-null  float64
 16  credit          26440 non-null  float64
 17  YEARS_BIRTH     26440 non-null 

# category / numerical columns

In [7]:
drop_c=['work_phone','phone','email','YEARS_EMPLOYED','id']
train.drop(drop_c,axis=1,inplace=True)
test.drop(drop_c,axis=1,inplace=True)

In [8]:
numerical_feats = train.dtypes[train.dtypes != "object"].index.tolist()
numerical_feats.remove('credit')
print("Number of Numerical features: ", len(numerical_feats))

categorical_feats = train.dtypes[train.dtypes == "object"].index.tolist()
print("Number of Categorical features: ", len(categorical_feats))

Number of Numerical features:  7
Number of Categorical features:  9


In [9]:
numerical_feats

['child_num',
 'income_total',
 'family_size',
 'begin_month',
 'YEARS_BIRTH',
 'Age_group',
 'income_avg']

In [10]:
categorical_feats

['gender',
 'car',
 'reality',
 'income_type',
 'edu_type',
 'family_type',
 'house_type',
 'occyp_type',
 'EMPLOYED_group']

In [14]:
col=train.dtypes.index.tolist()

In [19]:
print(list(train['gender'].value_counts().index))

['F', 'M']


In [24]:
for i in col:
    print(list(train[i].value_counts().index))
    #print(f'{i} 컬럼 항목은 : {list(train[i].value_counts().index)} 입니다.')

['F', 'M']
['N', 'Y']
['Y', 'N']
[0, 1, 2, 3, 4, 5]
[2632.5, 3071.25, 3510.0, 2193.75, 4387.5, 3948.75, 1755.0, 5265.0, 6142.5, 1316.25, 4826.25, 7020.0, 5703.75, 2457.0, 1579.5, 2369.25, 2808.0, 1930.5, 8775.0, 1491.75, 2018.25, 4212.0, 6581.25, 7897.5, 2895.75, 2281.5, 3334.5, 3246.75, 2106.0, 3159.0, 1404.0, 3685.5, 3773.25, 1842.75, 2544.75, 4563.0, 2983.5, 1053.0, 7458.75, 2720.25, 4914.0, 877.5, 13162.5, 4124.25, 4036.5, 10530.0, 3422.25, 1667.25, 5089.5, 5967.0, 3861.0, 3597.75, 1228.5, 5001.75, 1140.75, 8336.25, 965.25, 4650.75, 7634.25, 4738.5, 5177.25, 5791.5, 9652.5, 4475.25, 10968.75, 6318.0, 7546.5, 17550.0, 6844.5, 789.75, 5528.25, 4299.75, 12285.0, 5440.5, 1096.875, 6405.75, 2325.375, 6493.5, 5352.75, 2149.875, 3615.3, 6054.75, 3264.3, 15795.0, 614.25, 5879.25, 11407.5, 8687.25, 14040.0, 5616.0, 1184.625, 7283.25, 2386.8, 921.375, 11934.0, 7371.0, 9213.75, 2237.625, 2413.125, 3027.375, 1447.875, 2562.3, 1430.325, 1272.375, 649.35, 10924.875, 745.875, 8950.5, 1772.55, 434

# Data scaling

## income -> log scale

In [ ]:
for df in [train,test]:
    df['income_total'] = np.log1p(1+df['income_total'])

## Ordinal Encoder

In [ ]:
encoder = OrdinalEncoder(categorical_feats)
train[categorical_feats] = encoder.fit_transform(train[categorical_feats], train['credit'])
test[categorical_feats] = encoder.transform(test[categorical_feats])

#train['id'] = train['id'].astype('int64')
#test['id'] = test['id'].astype('int64')

## Clustering

In [ ]:
kmeans_train = train.drop(['credit'], axis=1)
kmeans = KMeans(n_clusters=36, random_state=42).fit(kmeans_train)
train['cluster'] = kmeans.predict(kmeans_train)
test['cluster'] = kmeans.predict(test)

## StandardScaler

In [ ]:
numerical_feats.remove('income_total')
scaler = StandardScaler()
train[numerical_feats] = scaler.fit_transform(train[numerical_feats])
test[numerical_feats] = scaler.transform(test[numerical_feats])

In [ ]:
train.info()

In [ ]:
train.head()

# MODELING

## CatBoostClassifier

In [ ]:
seed = 42
#n_fold_list = list(range(7,16,1))
n_fold_list = [14]
n_class = 3

target = 'credit'
X = train.drop(target, axis=1)
y = train[target]
X_test = test

In [ ]:
for n_fold in n_fold_list:
    print('***',n_fold,'***')

    skfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
    folds=[]
    for train_idx, valid_idx in skfold.split(X, y):
            folds.append((train_idx, valid_idx))

    cat_pred = np.zeros((X.shape[0], n_class))
    cat_pred_test = np.zeros((X_test.shape[0], n_class))
    cat_cols = ['gender', 'car', 'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type', 'EMPLOYED_group']
    #cat_cols = ['id',  'reality', 'income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type', 'YEARS_EMPLOYED']
    for fold in range(n_fold):
        print(f'\n----------------- Fold {fold} -----------------\n')
        train_idx, valid_idx = folds[fold]
        X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
        train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
        valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

        model_cat = CatBoostClassifier()
        model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)

        cat_pred[valid_idx] = model_cat.predict_proba(X_valid)
        cat_pred_test += model_cat.predict_proba(X_test) / n_fold
        print(f'CV Log Loss Score: {log_loss(y_valid, cat_pred[valid_idx]):.6f}')
        
    print(f'\t<*><*><*><*><*><*>Log Loss: {log_loss(y, cat_pred):.6f}<*><*><*><*><*><*>\n\n\n\n')

In [ ]:
def plot_feature_importance(importance,names,model_type):
    
    feature_importance = np.array(importance)
    feature_names = np.array(names)
    
    data={'feature_names':feature_names,'feature_importance':feature_importance}
    fi_df = pd.DataFrame(data)
    
    fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

    plt.figure(figsize=(10,8))

    sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])

    plt.title(model_type + ' Feature Importance')
    plt.xlabel('Feature Importance')
    plt.ylabel('Feature Names')

In [ ]:
plot_feature_importance(model_cat.get_feature_importance(),X_test.columns,'CATBOOST')

In [ ]:
# sub = pd.read_csv('./data/sample_submission.csv')
# sub.iloc[:, 1:] = cat_pred_test
# sub


In [ ]:
# sub.to_csv('submission0523_Final.csv', index=False)